Which optional argument in sql2csv will print detailed tracebacks and logs when errors occur while using sql2csv?


- `sql2csv -v` or `sql2csv --verbose`


Could you use csvsql's manual to check what SQL database connections are currently NOT supported for sql2csv and for the rest of the csvkit suite?

- ` csvsql -h | grep "sql"`

- Use sql2csv to access the SQLite database SpotifyDatabase and query and print all data in the table Spotify_Popularity.


```
# Verify database name 
ls

# Pull the entire Spotify_Popularity table and print in log
sql2csv --db "sqlite:///SpotifyDatabase.db" \
        --query "SELECT * FROM Spotify_Popularity" 
```

- Use a SQL query to print the first 5 rows in the table Spotify_Popularity. Then, preview the results using csvlook.

```
# Verify database name 
ls

# Query first 5 rows of Spotify_Popularity and print in log
sql2csv --db "sqlite:///SpotifyDatabase.db" \
        --query "SELECT * FROM Spotify_Popularity" \
        | csvlook         
```

- Save queried results to a new file Spotify_Popularity_5Rows.csv. Verify and preview the file with ls and csvlook.

```
# Verify database name 
ls

# Save query to new file Spotify_Popularity_5Rows.csv
sql2csv --db "sqlite:///SpotifyDatabase.db" \
        --query "SELECT * FROM Spotify_Popularity LIMIT 5" \
        > Spotify_Popularity_5Rows.csv

# Verify newly created file
ls

# Print preview of newly created file
csvlook Spotify_Popularity_5Rows.csv
```

- Complete the command to apply the SQL query to Spotify_MusicAttributes.csv.

```
# Preview CSV file
ls

# Apply SQL query to Spotify_MusicAttributes.csv
csvsql --query "SELECT * FROM Spotify_MusicAttributes ORDER BY duration_ms LIMIT 1" Spotify_MusicAttributes.csv
```

- Further improve the output by piping the output to csvlook.

```
# Reformat the output using csvlook 
csvsql --query "SELECT * FROM Spotify_MusicAttributes ORDER BY duration_ms LIMIT 1" \
	Spotify_MusicAttributes.csv | csvlook
```

- Instead of printing to console, re-direct output and save as new file: ShortestSong.csv.

```
# Re-direct output to new file: ShortestSong.csv
csvsql --query "SELECT * FROM Spotify_MusicAttributes ORDER BY duration_ms LIMIT 1" \
	Spotify_MusicAttributes.csv > ShortestSong.csv
    
# Preview newly created file 
csvlook ShortestSong.csv
```

- Fill in the csvsql command by calling upon the bash variable containing the SQL query instead of writing out the SQL query in full.

```
# Preview CSV file
ls

# Store SQL query as shell variable
sqlquery="SELECT * FROM Spotify_MusicAttributes ORDER BY duration_ms LIMIT 1"

# Apply SQL query to Spotify_MusicAttributes.csv
csvsql --query "$sqlquery" Spotify_MusicAttributes.csv
```

- Explore the data with the commands we have learned so far (e.g. csvstat, csvlook, etc). What is the column that Spotify_MusicAttributes.csv and Spotify_Popularity.csv have in common that can be used as the JOIN key?

```
csvcut -n Spotify_MusicAttributes.csv
csvcut -n Spotify_Popularity.csv
```

- Join Spotify_MusicAttributes.csv and Spotify_Popularity.csv together to form a new file Spotify_FullData.csv.

```
# Store SQL query as shell variable
sql_query="SELECT ma.*, p.popularity FROM Spotify_MusicAttributes ma INNER JOIN Spotify_Popularity p ON ma.track_id = p.track_id"

# Join 2 local csvs into a new csv using the saved SQL
csvsql --query "$sql_query" Spotify_MusicAttributes.csv Spotify_Popularity.csv > Spotify_FullData.csv

# Preview newly created file
csvstat Spotify_FullData.csv
```

- Upload Spotify_MusicAttributes.csv as its own table in the SQLite database SpotifyDatabase.
- Re-pull the data from the newly created table Spotify_MusicAttributes in the SQLite database SpotifyDatabase.

```
# Preview file
ls

# Upload Spotify_MusicAttributes.csv to database
csvsql --db "sqlite:///SpotifyDatabase.db" --insert Spotify_MusicAttributes.csv

# Store SQL query as shell variable
sqlquery="SELECT * FROM Spotify_MusicAttributes"

# Apply SQL query to re-pull new table in database
sql2csv --db "sqlite:///SpotifyDatabase.db" --query "$sqlquery" 
```

- Download the entire table SpotifyMostRecentData from the SQLite database SpotifyDatabase and save it as a csv file locally as SpotifyMostRecentData.csv.
- Manipulate the two local csv files SpotifyMostRecentData.csv and Spotify201812.csv by passing in the stored UNION ALL SQL query into csvsql. Save the newly created file as UnionedSpotifyData.csv
- Push the newly created csv file UnionedSpotifyData.csv back to database SpotifyDatabase as its own table.

```
# Store SQL for querying from SQLite database 
sqlquery_pull="SELECT * FROM SpotifyMostRecentData"

# Apply SQL to save table as local file 
sql2csv --db "sqlite:///SpotifyDatabase.db" --query "$sqlquery_pull" > SpotifyMostRecentData.csv

# Store SQL for UNION of the two local CSV files
sqlquery_union="SELECT * FROM SpotifyMostRecentData UNION ALL SELECT * FROM Spotify201812"

# Apply SQL to union the two local CSV files and save as local file
csvsql 	--query "$sqlquery_union" SpotifyMostRecentData.csv Spotify201812.csv > UnionedSpotifyData.csv

# Push UnionedSpotifyData.csv to database as a new table
csvsql --db "sqlite:///SpotifyDatabase.db" --insert UnionedSpotifyData.csv
```